# 학습 데이터를 카테고리에 맞게 재배치
- `ImageFolder` 활용하기 위해 학습 이미지를 새로운 폴더로 재배치(copy&paste)
- 레이블링 기준은 [여기](http://boostcamp.stages.ai/competitions/1/data/description)에서 확인할 수 있다
    
    

In [1]:
import os
from glob import glob
from tqdm import tqdm
import shutil
import pandas as pd

In [2]:
# 카테고리별 메타 정보 생성
masks = ['mask', 'incorrect', 'normal'] # 정착용 / 불완전착용 / 미착용
genders = ['male', 'female'] # 남성 / 여성
age_groups = ['young', 'middle', 'old'] # 30대 미만 / 30대 이상 60대 미만 / 60대 이상

class_meta = []
for m in masks:
    for g in genders:
        for a in age_groups:
            class_meta.append((m, g, a))

class_meta

[('mask', 'male', 'young'),
 ('mask', 'male', 'middle'),
 ('mask', 'male', 'old'),
 ('mask', 'female', 'young'),
 ('mask', 'female', 'middle'),
 ('mask', 'female', 'old'),
 ('incorrect', 'male', 'young'),
 ('incorrect', 'male', 'middle'),
 ('incorrect', 'male', 'old'),
 ('incorrect', 'female', 'young'),
 ('incorrect', 'female', 'middle'),
 ('incorrect', 'female', 'old'),
 ('normal', 'male', 'young'),
 ('normal', 'male', 'middle'),
 ('normal', 'male', 'old'),
 ('normal', 'female', 'young'),
 ('normal', 'female', 'middle'),
 ('normal', 'female', 'old')]

In [19]:

def get_class(folder_name: str, img_name: str) -> int:
    """
    18개 카테고리 중 해당되는 클래스 인덱스를 리턴

    Reference.
        - http://boostcamp.stages.ai/competitions/1/data/description
    """
    gender = 'female' if 'female' in folder_name else 'male'

    age = int(folder_name.split('_')[-1])
    if age < 30:
        age_group = 'young' # 30대 미만
    elif age >= 60:
        age_group = 'old' # 60대 이상
    else:
        age_group = 'middle' # 그 외

    if 'incorrect' in img_name: # 불완전 착용
        mask_state = 'incorrect'
    elif 'normal' in img_name: # 미착용
        mask_state = 'normal'
    else: # 정착용
        mask_state = 'mask'

    info = (mask_state, gender, age_group)
    class_ = class_meta.index(info) # 클래스 추출
    return class_


In [11]:
# 카테고리별 이미지 폴더를 생성
# 이미지를 이동/복사할 디렉토리
SAVE_DIR = '../preprocessed/train'

try:
    os.listdir(SAVE_DIR)
except:
    os.mkdir(SAVE_DIR)

# 18 개 클래스 생성
for i in range(18):
    if str(i) not in os.listdir(SAVE_DIR):
        os.mkdir(os.path.join(SAVE_DIR, str(i)))

In [22]:
TRAIN_DIR = '../input/data/train/images/' # 기존 학습 데이터 디렉토리

for folder in tqdm(glob(os.path.join(TRAIN_DIR, '*')), desc='Classify Train Data'):
    folder_name = os.path.basename(folder)
    for img in glob(os.path.join(folder, '*')):
        img_name = os.path.basename(img)
        class_ = get_class(folder_name, img_name) # 이미지의 클래스 추출
        print(class_, folder_name, img_name)
        copy_file_path = os.path.join(SAVE_DIR, str(class_), f'{folder_name}_{img_name}')
        shutil.copy(img, copy_file_path)

ng
5 003827_female_Asian_60 mask4.jpg
17 003827_female_Asian_60 normal.jpg
5 003827_female_Asian_60 mask3.jpg
5 003827_female_Asian_60 mask1.jpg
11 003827_female_Asian_60 incorrect_mask.jpg
5 003827_female_Asian_60 mask5.jpg
5 003827_female_Asian_60 mask2.jpg
3 003306_female_Asian_20 mask4.jpg
15 003306_female_Asian_20 normal.jpg
3 003306_female_Asian_20 mask3.jpg
3 003306_female_Asian_20 mask1.jpg
9 003306_female_Asian_20 incorrect_mask.jpg
3 003306_female_Asian_20 mask5.jpg
3 003306_female_Asian_20 mask2.jpg
0 001144_male_Asian_28 mask4.jpg
12 001144_male_Asian_28 normal.jpg
0 001144_male_Asian_28 mask3.jpg
0 001144_male_Asian_28 mask1.jpg
6 001144_male_Asian_28 incorrect_mask.jpg
0 001144_male_Asian_28 mask5.jpg
0 001144_male_Asian_28 mask2.jpg
3 001289_female_Asian_23 mask4.jpg
15 001289_female_Asian_23 normal.jpg
3 001289_female_Asian_23 mask3.jpg
3 001289_female_Asian_23 mask1.jpg
9 001289_female_Asian_23 incorrect_mask.jpg
3 001289_female_Asian_23 mask5.jpg
3 001289_female_Asian

In [6]:
# 복사가 잘 되었는지 확인
total_files = 0
for dir_ in glob(os.path.join(SAVE_DIR, '*')):
    num_files = len(os.listdir(dir_))
    class_ = os.path.basename(dir_)
    print(f'파일 수: {num_files}, 클래스: {class_}')
    total_files += num_files
print('전체 데이터:', total_files)

파일 수: 2050, 클래스: 1
파일 수: 732, 클래스: 15
파일 수: 4085, 클래스: 4
파일 수: 410, 클래스: 13
파일 수: 415, 클래스: 2
파일 수: 109, 클래스: 11
파일 수: 83, 클래스: 14
파일 수: 549, 클래스: 6
파일 수: 83, 클래스: 8
파일 수: 545, 클래스: 5
파일 수: 109, 클래스: 17
파일 수: 410, 클래스: 7
파일 수: 817, 클래스: 10
파일 수: 2745, 클래스: 0
파일 수: 549, 클래스: 12
파일 수: 3660, 클래스: 3
파일 수: 817, 클래스: 16
파일 수: 732, 클래스: 9
전체 데이터: 18900
